In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: C:\Users\James\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [356]:
#  Set up a relative path
import  os
print(os.getcwd())
filename = "BrianC.pdf"
file = os.path.join(os.path.abspath('..'), f'heat_sheets\{filename}')



c:\Users\James\Documents\School\CS\tbd2\experiment_notebooks


In [357]:
# importing required modules 
from PyPDF2 import PdfReader 
# creating a pdf reader object 
reader = PdfReader(file) 
# Addr all the pages to one string
raw_text = ''
for page in reader.pages:
    raw_text += page.extract_text() 

Clean the text before parsing

In [358]:
import re

# Use regex to replace multiple consecutive spaces with a single space
def remove_redudant_spaces(input_string):
    cleaned_string = re.sub(r'[ ]+', ' ', input_string)
    return cleaned_string

# Use regex to find a number followed by a letter and add a newline
# This is a common PDF error
def add_newline_after_number(input_string):
    pattern = re.compile(r'(\d)([A-Za-z])')
    result = pattern.sub(r'\1\n\2', input_string)
    return result

# Use regex to fix another common PDF error
def add_newline_before_keyword(keywords_list, text):
    for keyword in keywords_list:
        pattern = re.compile(rf'(?<=.)({re.escape(keyword)})', re.IGNORECASE)
        modified_text = pattern.sub(r'\n\1', text)
        text = modified_text
    return modified_text

cleaned_text = remove_redudant_spaces(raw_text)
cleaned_text = add_newline_after_number(cleaned_text)
cleaned_text = add_newline_before_keyword(["Event","Yr"], cleaned_text) # Fix instances of [word]Event or [word]Yr
print(cleaned_text)

Finished Results - Contractor License Hy-Tek's MEET MANAGER 10:13 PM 4/5/2023 Page 12023 Bryan Clay Invitational - 4/13/2023 to 4/15/2023
Hosted by: Azusa Pacific UniversityCougar StadiumPerformance List
Event 2 Women 10000 Meter Run
Yr 
 School Name Seed Time
George Fox FR Aalpoel, Kayla 1
Liberty JR Ackley-Fairley, Adelyn 2
Saginaw Vall JR Amthor, Lynsey 3
UC Irvine SO Andar, Sameen 4
Westminster SR Anderson, Saydi 5
Guatemala Aroche Lopez, Viviana 6
Northwestern JR Bartolotto, Kalea 7
British Colu JR Becker, Kyla 8
British Colu FR Beckett, Marley 9
Butler SO Bertrand, Laure 10
UC San Diego JR Billman, Caitlin 11
Claremont-Mu SR Bloss, Meredith 12
Sonoma St. FR Bomarito, Gianna 13
Weber State JR Brow, Danielle 14
Dallas Bapti SR Brown, Madison 15
Black Hills SO Brown, Sylvia 16
Cal St. Full JR Caesar, McKaylie 17
UAB JR Charlton, Lauren 18
Alabama SR Chelangat, Mercy 19
Toledo FR Chirchir, Joy 20
Nevada JR Clarke, Emily 21
Iowa State JR Cohoon, Brenna 22
Northern Ken SR Cole, Megan 2

Use regex to identify and group events

In [359]:
# Define regex to find each event in the format (Event)(Number)(Gender)(Distance)(Meters)(Type)
event_pattern = re.compile(r"(Event).*?(\d+)[^\n]*\n{0,2}?(Women|Men).*?(\d+)[^\n]*\n{0,2}?(Meter)[^\n]*\n{0,2}?(Steeplechase|Run)")

# find the start and end index of each event within the string
# will use this to divide the pdf text up into different sections by event
split_points = []
iter = event_pattern.finditer(cleaned_text)
for match in iter:
    split_points.append((match.start(),match.end()))

# add events to dictionary
#TODO there should probably be a "runners" field of each event
events = []
event_matches = event_pattern.findall(cleaned_text)
for match in event_matches:
    events.append({
        "event_number": match[1],
        "gender":match[2],
        "distance":match[3],
        "type":match[5],
        "athletes_list": []
        })
print(events)


[{'event_number': '2', 'gender': 'Women', 'distance': '10000', 'type': 'Run', 'athletes_list': []}, {'event_number': '2', 'gender': 'Women', 'distance': '10000', 'type': 'Run', 'athletes_list': []}, {'event_number': '3', 'gender': 'Men', 'distance': '10000', 'type': 'Run', 'athletes_list': []}, {'event_number': '3', 'gender': 'Men', 'distance': '10000', 'type': 'Run', 'athletes_list': []}, {'event_number': '5', 'gender': 'Men', 'distance': '3000', 'type': 'Steeplechase', 'athletes_list': []}, {'event_number': '5', 'gender': 'Men', 'distance': '3000', 'type': 'Steeplechase', 'athletes_list': []}, {'event_number': '5', 'gender': 'Men', 'distance': '3000', 'type': 'Steeplechase', 'athletes_list': []}, {'event_number': '6', 'gender': 'Women', 'distance': '3000', 'type': 'Steeplechase', 'athletes_list': []}, {'event_number': '6', 'gender': 'Women', 'distance': '3000', 'type': 'Steeplechase', 'athletes_list': []}, {'event_number': '7', 'gender': 'Men', 'distance': '5000', 'type': 'Run', 'ath

In [360]:
# This function splits the text at different ranges
# The ranges correspond to the location of events within the text
def split_text_at_ranges(input_string, split_ranges):
    result = []
    start_index = 0

    for split_range in split_ranges:
        if isinstance(split_range, tuple) and len(split_range) == 2:
            split_start, split_end = split_range

            if 0 <= split_start < split_end <= len(input_string):
                # Add the part before the range
                result.append(input_string[start_index:split_start])

                # Move the start index to the end of the range
                start_index = split_end

    # Append the remaining part of the string
    result.append(input_string[start_index:])

    return result

In [361]:
split_string = split_text_at_ranges(cleaned_text,split_points)
# Print statement to show the spliting is working 
for line in split_string[1:]:
    print("###############")
    print(line)
    print("###############")

###############

Yr 
 School Name Seed Time
George Fox FR Aalpoel, Kayla 1
Liberty JR Ackley-Fairley, Adelyn 2
Saginaw Vall JR Amthor, Lynsey 3
UC Irvine SO Andar, Sameen 4
Westminster SR Anderson, Saydi 5
Guatemala Aroche Lopez, Viviana 6
Northwestern JR Bartolotto, Kalea 7
British Colu JR Becker, Kyla 8
British Colu FR Beckett, Marley 9
Butler SO Bertrand, Laure 10
UC San Diego JR Billman, Caitlin 11
Claremont-Mu SR Bloss, Meredith 12
Sonoma St. FR Bomarito, Gianna 13
Weber State JR Brow, Danielle 14
Dallas Bapti SR Brown, Madison 15
Black Hills SO Brown, Sylvia 16
Cal St. Full JR Caesar, McKaylie 17
UAB JR Charlton, Lauren 18
Alabama SR Chelangat, Mercy 19
Toledo FR Chirchir, Joy 20
Nevada JR Clarke, Emily 21
Iowa State JR Cohoon, Brenna 22
Northern Ken SR Cole, Megan 23
Bowling Gree SO Collett, Olivia 24
Msu Denver JR Cornell, Caitlin 25
Louisiana SR Daigle, Sophie 26
Wyoming FR Dalbec, Eowyn 27
Caltech SO Dalfonzo, Sophia 28
Pepperdine SR Danley, Skyler 29
New Mexico JR Dishon, Sa

We now have a list of events and the text of a pdf heat sheet split by event. What's left is to extract the runners from each event.

In [362]:
# Regular expression patterns for extracting runner information 
# runner info in the format of (School Name)(School Year)(Last, First)(Seed)
runner_pattern = re.compile(r"((?:[A-Za-z]-?[ ]?)+)\s*([A-ZA-Z]{2})\s*([A-Za-z]+, [A-Za-z]+)\s*(\d+)")

runners_list = []
event_count = 0
for section in split_string[1:]:
    athletes_list = []
    current_event = events[event_count] # find the event the runners are in
    athlete_matches = runner_pattern.findall(section) # find all regex matches
    # create an athlete dictionary object
    for athlete in athlete_matches:
        athlete_dict = {
            "school":athlete[0],
            "year":athlete[1],
            "name":athlete[2],
            "seed":athlete[3]
        }
        athletes_list.append(athlete_dict)
    # update the event with the list of athletes
    current_event["athletes_list"] = athletes_list
    event_count+=1


In [368]:
# patterns for identifying athlete info 
name_pattern = re.compile(r"([A-Za-z]+, [A-Za-z]+)")
school_pattern = re.compile(r"((?:[A-Za-z]-?[ ]?)+)")
year_pattern = re.compile(r"FR|SO|JR|SR|Gr")
time_pattern = r'\b(?:\d{1,2}:)?\d{1,2}\.\d{1,2}\b'
seed_pattern = re.compile(r"\d+")

# get time. seed, then year, then name, then school,
def find_athlete_in_line(line):
    if line == '':
        return None
    match_count = 0
    modified_text = line

    time = re.findall(time_pattern,modified_text)
    modified_text = re.sub(time_pattern,"",modified_text) # remove pattern from text after finding it

    seed = re.findall(seed_pattern,modified_text) # There should only be one seed time anyway
    modified_text = re.sub(seed_pattern,"",modified_text)

    year = re.findall(year_pattern,modified_text)
    modified_text = re.sub(year_pattern,"",modified_text)

    name = re.findall(name_pattern,modified_text)
    modified_text = re.sub(name_pattern,"",modified_text)

    school = re.findall(school_pattern,modified_text)
    modified_text = re.sub(school_pattern,"",modified_text)
    
    school = None if "Stadium" in school else school # this filters out stadium records
    
    if time: match_count+=1 
    if seed: match_count+=1
    if year: match_count+=1
    if name: match_count+=1
    if school: match_count+=1 
    if match_count < 4:
        return None
    return({
        "name":name[0] if name else None,
        "year":year[0] if year else None,
        "school":school[0] if school else None,
        "time":time[0] if time else None,
        "seed":seed[0] if seed else None
    })

In [369]:
# Regular expression patterns for extracting runner information 
# runner info in the format of (School Name)(School Year)(Last, First)(Seed)
runner_pattern = re.compile(r"((?:[A-Za-z]-?[ ]?)+)\s*([A-ZA-Z]{2})\s*([A-Za-z]+, [A-Za-z]+)\s*(\d+)")

runners_list = []
event_count = 0
for section in split_string[1:]:
    athletes_list = []
    current_event = events[event_count] # find the event the runners are in
    for line in section.split("\n"):
       athlete_dict = find_athlete_in_line(line)
       athletes_list.append(athlete_dict) if athlete_dict else None
    # update the event with the list of athletes
    current_event["athletes_list"] = athletes_list
    event_count+=1


In [370]:
print(events)

[{'event_number': '2', 'gender': 'Women', 'distance': '10000', 'type': 'Run', 'athletes_list': [{'name': 'Aalpoel, Kayla', 'year': 'FR', 'school': 'George Fox ', 'time': None, 'seed': '1'}, {'name': 'Fairley, Adelyn', 'year': 'JR', 'school': 'Liberty ', 'time': None, 'seed': '2'}, {'name': 'Amthor, Lynsey', 'year': 'JR', 'school': 'Saginaw Vall ', 'time': None, 'seed': '3'}, {'name': 'Andar, Sameen', 'year': 'SO', 'school': 'UC Irvine ', 'time': None, 'seed': '4'}, {'name': 'Anderson, Saydi', 'year': 'SR', 'school': 'Westminster ', 'time': None, 'seed': '5'}, {'name': 'Bartolotto, Kalea', 'year': 'JR', 'school': 'Northwestern ', 'time': None, 'seed': '7'}, {'name': 'Becker, Kyla', 'year': 'JR', 'school': 'British Colu ', 'time': None, 'seed': '8'}, {'name': 'Beckett, Marley', 'year': 'FR', 'school': 'British Colu ', 'time': None, 'seed': '9'}, {'name': 'Bertrand, Laure', 'year': 'SO', 'school': 'Butler ', 'time': None, 'seed': '10'}, {'name': 'Billman, Caitlin', 'year': 'JR', 'school':

Save it all to a json

In [371]:
import json
def save_list_to_json(data_list, filename):
    with open(filename, 'w') as json_file:
        json.dump(data_list, json_file,indent=2)

save_list_to_json(events,"outputs/brianC_results.json")